## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Namatanai,PG,-3.6667,152.4333,85.05,69,82,13.80,broken clouds
1,1,Tuy Hoa,VN,13.0833,109.3000,79.77,70,43,4.27,scattered clouds
2,2,Bonthe,SL,7.5264,-12.5050,78.66,83,99,2.17,overcast clouds
3,3,Tiksi,RU,71.6872,128.8694,7.56,85,100,9.46,overcast clouds
4,4,Upernavik,GL,72.7868,-56.1549,4.17,88,100,8.12,overcast clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 100


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Namatanai,PG,-3.6667,152.4333,85.05,69,82,13.80,broken clouds
1,1,Tuy Hoa,VN,13.0833,109.3000,79.77,70,43,4.27,scattered clouds
2,2,Bonthe,SL,7.5264,-12.5050,78.66,83,99,2.17,overcast clouds
5,5,Atuona,PF,-9.8000,-139.0333,80.73,73,64,19.73,broken clouds
6,6,Vaini,TO,-21.2000,-175.2000,80.60,69,20,5.75,few clouds
9,9,Butaritari,KI,3.0707,172.7902,82.29,77,98,9.04,light rain
11,11,Hobart,AU,-42.8794,147.3294,62.01,55,75,16.11,broken clouds
14,14,Saint-Georges,GD,12.0564,-61.7485,80.60,78,20,19.57,few clouds
16,16,Katobu,ID,-4.9408,122.5276,84.34,66,95,3.47,overcast clouds
20,20,Yenagoa,NG,4.9247,6.2642,77.74,91,100,3.09,overcast clouds


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather Description    0
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Namatanai,PG,85.05,broken clouds,-3.6667,152.4333,
1,Tuy Hoa,VN,79.77,scattered clouds,13.0833,109.3000,
2,Bonthe,SL,78.66,overcast clouds,7.5264,-12.5050,
5,Atuona,PF,80.73,broken clouds,-9.8000,-139.0333,
6,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,
9,Butaritari,KI,82.29,light rain,3.0707,172.7902,
11,Hobart,AU,62.01,broken clouds,-42.8794,147.3294,
14,Saint-Georges,GD,80.60,few clouds,12.0564,-61.7485,
16,Katobu,ID,84.34,overcast clouds,-4.9408,122.5276,
20,Yenagoa,NG,77.74,overcast clouds,4.9247,6.2642,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping city")
        

Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city
Hotel not found... skipping city


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']


# confirm the dataframe looks like example provided in module
clean_hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Namatanai,PG,85.05,broken clouds,-3.6667,152.4333,Namatanai Hotel
1,Tuy Hoa,VN,79.77,scattered clouds,13.0833,109.3000,CenDeluxe Hotel Tuy Hoa
2,Bonthe,SL,78.66,overcast clouds,7.5264,-12.5050,Bonthe Holiday Village
5,Atuona,PF,80.73,broken clouds,-9.8000,-139.0333,Villa Enata
6,Vaini,TO,80.60,few clouds,-21.2000,-175.2000,Keleti Beach Resort
9,Butaritari,KI,82.29,light rain,3.0707,172.7902,Isles Sunset Lodge
11,Hobart,AU,62.01,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
14,Saint-Georges,GD,80.60,few clouds,12.0564,-61.7485,Radisson Grenada Beach Resort
16,Katobu,ID,84.34,overcast clouds,-4.9408,122.5276,La Ode Husein
20,Yenagoa,NG,77.74,overcast clouds,4.9247,6.2642,Aridolf Resort Wellness and Spa


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 

locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))